In [11]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import numpy as np
from numpy import zeros_like
from IPython.display import display
def display_matrix(m):
    display(sympy.Matrix(m))
import sympy
import pandas as pd
import scipy.stats as ss

sympy.init_printing()

In [12]:
basis = 'leguerre'

# parameters

mu = 0.06
r = 0.06
T = 1
I = 250
paths = 10000
S0 = 40
S0_list = range(37, 43, 1)
m = 0
K = 40

GBM_sigma = 0.04
GBM_mu = r

LN_lam = 1
LN_sigma = 0.02
LN_mu = r
LN_v = 0.02

JR_lam = 0.01
JR_sigma = 0.03
JR_mu = r + JR_lam

LNparams = (LN_lam, LN_sigma, LN_mu, LN_v, m)
JRparams = (JR_lam, JR_sigma, JR_mu)
GBMparams = (GBM_mu, GBM_sigma)

In [13]:
def merton_jump_to_ruin_paths_sample(S0, paths, I, T, lam, sigma, mu):
    S0, paths, I, T = S0, paths, I, T
    matrix = np.zeros((paths, I))
    all_normals = []
    for k in range(paths):
        X = np.zeros(I)
        S = np.zeros(I)
        X[0] = np.log(S0)
        S[0] = S0
        dt = T / I
        for i in range(1,I):
            Z = np.random.standard_normal()
            all_normals.append(Z)
            N = np.random.poisson(lam * dt)
            if N == 0 and X[i-1] > 0:
                X[i] = X[i-1] + (mu - 0.5 * sigma ** 2) * dt + sigma * np.sqrt(dt) * Z
                S[i] = np.exp(X[i])
            else:
                S[i] = 0
        matrix[k] = S
    mean_normal = np.mean(all_normals)
    std_normal = np.std(all_normals)
    return mean_normal, std_normal

In [14]:
mean_normal_JR, std_normal_JR = merton_jump_to_ruin_paths_sample(S0, paths, I, T, *JRparams)

In [15]:
def merton_jump_to_ruin_paths_mm(S0, paths, I, T, lam, sigma, mu, mean_normal_JR, std_normal_JR):
    S0, paths, I, T = S0, paths, I, T
    matrix = np.zeros((paths, I))
    for k in range(paths):
        X = np.zeros(I)
        S = np.zeros(I)
        X[0] = np.log(S0)
        S[0] = S0
        dt = T / I
        for i in range(1,I):
            Z = np.random.standard_normal()
            N = np.random.poisson(lam * dt)
            if N == 0 and X[i-1] > 0:
                X[i] = X[i-1] + (mu - 0.5 * sigma ** 2) * dt + sigma * np.sqrt(dt) * (Z - mean_normal_JR) / std_normal_JR
                S[i] = np.exp(X[i])
            else:
                S[i] = 0
        matrix[k] = S
    return matrix

In [16]:
def LSM(K, S, I, df, basis, deg):
    paths = len(S)
    np.random.seed(42)
    H = np.maximum(K - S, 0)  # intrinsic values for put option
    V = np.zeros_like(H)  # value matrix
    V[:, -1] = H[:, -1]  # set value at maturity equal to intrinsic value

    # Valuation by LS Method
    for t in range(I - 2, 0, -1):
        good_paths = H[:, t] > 0  # paths where the intrinsic value is positive

        if np.sum(good_paths) > 0:
            if basis == 'poly':
                rg = np.polyfit(S[good_paths, t], V[good_paths, t + 1] * df, deg)
                C = np.polyval(rg, S[good_paths, t])
            elif basis == 'legendre':
                rg = np.polynomial.legendre.legfit(S[good_paths, t], V[good_paths, t + 1] * df, deg)
                C = np.polynomial.legendre.legval(S[good_paths, t], rg)
            elif basis =='laguerre':
                rg = np.polynomial.laguerre.lagfit(S[good_paths, t], V[good_paths, t + 1] * df, deg)
                C = np.polynomial.laguerre.lagval(S[good_paths, t], rg)
            else:  # 'hermite'
                rg = np.polynomial.hermite.hermfit(S[good_paths, t], V[good_paths, t + 1] * df, deg)
                C = np.polynomial.hermite.hermval(S[good_paths, t], rg)

            exercise = np.zeros(len(good_paths), dtype=bool)
            exercise[good_paths] = H[good_paths, t] > C
        else:
            # If all intrinsic values are zero, mark all as non-exercise
            exercise = np.zeros(len(good_paths), dtype=bool)

        V[exercise, t] = H[exercise, t]
        V[exercise, t + 1 :] = 0
        discount_path = ~exercise
        V[discount_path, t] = V[discount_path, t + 1] * df

    V0 = np.mean(V[:, 1]) * df  # discounted expectation of V[t=1]
    V0_array = V[:, 1] * df
    SE = np.std(V[:, 1] * df) / np.sqrt(paths)
    variance = np.var(V[:, 1] * df)
    return V0, V0_array, SE, variance

In [17]:
def merton_jump_to_ruin_paths(S0, paths, I, T, lam, sigma, mu):
    S0, paths, I, T = S0, paths, I, T
    matrix = np.zeros((paths, I))
    for k in range(paths):
        X = np.zeros(I)
        S = np.zeros(I)
        X[0] = np.log(S0)
        S[0] = S0
        dt = T / I
        for i in range(1,I):
            Z = np.random.standard_normal()
            N = np.random.poisson(lam * dt)
            if N == 0 and X[i-1] > 0:
                X[i] = X[i-1] + (mu - 0.5 * sigma ** 2) * dt + sigma * np.sqrt(dt) * Z
                S[i] = np.exp(X[i])
            else:
                S[i] = 0
        matrix[k] = S
    return matrix

In [18]:
JR_mm = merton_jump_to_ruin_paths_mm(S0, paths, I, T, *JRparams, mean_normal_JR, std_normal_JR)
JR = merton_jump_to_ruin_paths(S0, paths, I, T, *JRparams)

V0_JR_mm, V0_array_JR_mm, SE_JR_mm, variance_JR_mm = LSM(K, JR_mm, I, np.exp(-r * T), basis, 3)
V0_JR, V0_array_JR, SE_JR, variance_JR = LSM(K, JR, I, np.exp(-r * T), basis, 3)

print('JR_mm: V0 = {:.4f}, SE = {:.4f}, variance = {:.4f}'.format(V0_JR_mm, SE_JR_mm, variance_JR_mm))
print('JR: V0 = {:.4f}, SE = {:.4f}, variance = {:.4f}'.format(V0_JR, SE_JR, variance_JR))

JR_mm: V0 = 0.0647, SE = 0.0057, variance = 0.3259
JR: V0 = 0.0693, SE = 0.0067, variance = 0.4523
